In [9]:
import polars as pl
from kmodes.kmodes import KModes
import pickle

In [ ]:
users_dir = r"C:\Users\muthuvu\Videos\10X-MLE-master\data\users\part-00001.parquet"
events_dir = r"C:\Users\muthuvu\Videos\10X-MLE-master\data\events\part-00001.parquet"

# comment above 2 lines & un-comment the following 2 lines to read all files
#users_dir = "/home/muthu/Desktop/New Folder/data/users/*.parquet" 
#events_dir = "/home/muthu/Desktop/New Folder/data/events/*.parquet" 

users = pl.scan_parquet(users_dir)
events = pl.scan_parquet(events_dir)

users = users.drop(["email", "phone", "name", "last_active_at"])
events = events.drop(["event_id", "event_time", "page_url", "product_id", "traffic_source", "session_id"])

df = events.join(users, left_on="muid", right_on="id") 

In [2]:
small_df = df.head(20000).collect()
small_df

shape: (20_000, 6)
┌─────────────────────────────────┬─────────────┬───────────┬─────────┬──────────┬──────────┐
│ muid                            ┆ event_name  ┆ device_os ┆ channel ┆ category ┆ location │
│ ---                             ┆ ---         ┆ ---       ┆ ---     ┆ ---      ┆ ---      │
│ str                             ┆ str         ┆ str       ┆ str     ┆ str      ┆ str      │
╞═════════════════════════════════╪═════════════╪═══════════╪═════════╪══════════╪══════════╡
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ scroll      ┆ android   ┆ browser ┆ books    ┆ CA       │
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ add_to_cart ┆ android   ┆ browser ┆ beauty   ┆ CA       │
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ viewcontent ┆ android   ┆ browser ┆ books    ┆ CA       │
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ scroll      ┆ android   ┆ browser ┆ books    ┆ CA       │
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ pageview    ┆ android   ┆ browser ┆ books    ┆ CA       │
│ …                               ┆ …           ┆ …         ┆ …       ┆ …        ┆ …        │
│ bb74bbbb-15b6-42d3-bef7-5df88e… ┆ viewcontent ┆ android   ┆ app     ┆ coffee   ┆ PK       │
│ bb74bbbb-15b6-42d3-bef7-5df88e… ┆ pageview    ┆ android   ┆ app     ┆ books    ┆ PK       │
│ bb74bbbb-15b6-42d3-bef7-5df88e… ┆ search      ┆ android   ┆ app     ┆ laptop   ┆ PK       │
│ bb74bbbb-15b6-42d3-bef7-5df88e… ┆ viewcontent ┆ android   ┆ app     ┆ books    ┆ PK       │
│ bb74bbbb-15b6-42d3-bef7-5df88e… ┆ add_to_cart ┆ android   ┆ app     ┆ fashion  ┆ PK       │
└─────────────────────────────────┴─────────────┴───────────┴─────────┴──────────┴──────────┘

In [ ]:
small_df_pd = small_df.to_pandas().drop('muid', axis=1)
km = KModes(n_clusters=150, verbose=0)
small_df_pd["cluster_id"] = km.fit_predict(small_df_pd)
small_df_pd

event_name device_os  channel category location  cluster_id
0           scroll   android  browser    books       CA           0
1      add_to_cart   android  browser   beauty       CA          98
2      viewcontent   android  browser    books       CA          16
3           scroll   android  browser    books       CA           0
4         pageview   android  browser    books       CA          19
...            ...       ...      ...      ...      ...         ...
19995  viewcontent   android      app   coffee       PK          20
19996     pageview   android      app    books       PK          17
19997       search   android      app   laptop       PK          37
19998  viewcontent   android      app    books       PK          20
19999  add_to_cart   android      app  fashion       PK          97

[20000 rows x 6 columns]

In [6]:
with open(r"C:\Users\muthuvu\Videos\10X-MLE-master\result\kmodes_model.pkl", "wb") as f:
    pickle.dump(km, f)
small_df_pd.to_csv(r"C:\Users\muthuvu\Videos\10X-MLE-master\result\clustering_data.csv", index=False)

In [2]:

with open(r"C:\Users\muthuvu\Videos\10X-MLE-master\result\kmodes_model.pkl", "rb") as f:
    km_loaded = pickle.load(f)

#df["cluster_id"] = km_loaded.predict(df.collect())

NameError: name 'pickle' is not defined

In [3]:
import pandas as pd
small_df_pd = pd.read_csv(r"C:\Users\muthuvu\Videos\10X-MLE-master\result\clustering_data.csv"); small_df_pd

muid   event_name device_os  channel  \
0      f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
1      f2ffe5a9-165d-4021-8707-d197d7ce8e20  add_to_cart   android  browser   
2      f2ffe5a9-165d-4021-8707-d197d7ce8e20  viewcontent   android  browser   
3      f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
4      f2ffe5a9-165d-4021-8707-d197d7ce8e20     pageview   android  browser   
...                                     ...          ...       ...      ...   
19995  bb74bbbb-15b6-42d3-bef7-5df88e275e90  viewcontent   android      app   
19996  bb74bbbb-15b6-42d3-bef7-5df88e275e90     pageview   android      app   
19997  bb74bbbb-15b6-42d3-bef7-5df88e275e90       search   android      app   
19998  bb74bbbb-15b6-42d3-bef7-5df88e275e90  viewcontent   android      app   
19999  bb74bbbb-15b6-42d3-bef7-5df88e275e90  add_to_cart   android      app   

      category location  cluster_id  
0        books       CA          52  
1       beauty       CA          67  
2        books       CA          67  
3        books       CA          52  
4        books       CA         137  
...        ...      ...         ...  
19995   coffee       PK          95  
19996    books       PK         147  
19997   laptop       PK          95  
19998    books       PK          77  
19999  fashion       PK          80  

[20000 rows x 7 columns]

In [22]:
from dask.dataframe import from_pandas
df_dd = from_pandas(df.collect().to_pandas())
df_dd

Dask DataFrame Structure:
                 muid event_name device_os channel category location
npartitions=1                                                       
0              string     string    string  string   string   string
13020869          ...        ...       ...     ...      ...      ...
Dask Name: frompandas, 1 expression
Expr=df

In [12]:
def predict_partition(pdf):
    # Ensure numpy array (string type, no pd.NA)
    arr = pdf.astype(str).to_numpy()
    pdf["cluster"] = km_loaded.predict(arr)
    return pdf

X_dd = df_dd.drop("muid", axis=1)

# Apply partition-wise
result_dd = X_dd.map_partitions(predict_partition)
final_dd = df_dd[["muid"]].join(result_dd)

In [11]:
with open(r"C:\Users\muthuvu\Videos\10X-MLE-master\result\kmodes_model.pkl", "rb") as f:
    km_loaded = pickle.load(f)

from dask.dataframe import from_pandas
df_dd = from_pandas(df.collect().to_pandas())
df_dd

Dask DataFrame Structure:
                 muid event_name device_os channel category location
npartitions=1                                                       
0              string     string    string  string   string   string
13020869          ...        ...       ...     ...      ...      ...
Dask Name: frompandas, 1 expression
Expr=df

In [14]:
final_dd.repartition(npartitions=1).to_parquet(
    "output_clusters_single/", 
    engine="pyarrow", 
    write_index=False
)

In [ ]:
def summarize_cluster(df_cluster):
    device = df_cluster["device"].mode()[0]
    channel = df_cluster["channel"].mode()[0]
    category = df_cluster["category"].mode()[0]
    
    actions = df_cluster["event"].value_counts()
    top_action = actions.idxmax()
    
    if "purchase" not in actions:
        return f"{device} {channel} users viewing {category} but not purchasing"
    elif "add_to_cart" in actions and actions["add_to_cart"] > actions.get("purchase", 0):
        return f"{device} {channel} users adding to cart for {category}"
    else:
        return f"{device} {channel} users mostly {top_action} in {category}"
c

shape: (13_020_870, 6)
┌─────────────────────────────────┬─────────────┬───────────┬─────────┬──────────┬──────────┐
│ muid                            ┆ event_name  ┆ device_os ┆ channel ┆ category ┆ location │
│ ---                             ┆ ---         ┆ ---       ┆ ---     ┆ ---      ┆ ---      │
│ str                             ┆ str         ┆ str       ┆ str     ┆ str      ┆ str      │
╞═════════════════════════════════╪═════════════╪═══════════╪═════════╪══════════╪══════════╡
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ scroll      ┆ android   ┆ browser ┆ books    ┆ CA       │
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ add_to_cart ┆ android   ┆ browser ┆ beauty   ┆ CA       │
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ viewcontent ┆ android   ┆ browser ┆ books    ┆ CA       │
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ scroll      ┆ android   ┆ browser ┆ books    ┆ CA       │
│ f2ffe5a9-165d-4021-8707-d197d7… ┆ pageview    ┆ android   ┆ browser ┆ books    ┆ CA       │
│ …                               ┆ …           ┆ …         ┆ …       ┆ …        ┆ …        │
│ 85d63b71-2463-4097-adfd-012c23… ┆ pageview    ┆ android   ┆ browser ┆ books    ┆ ID       │
│ 85d63b71-2463-4097-adfd-012c23… ┆ viewcontent ┆ android   ┆ browser ┆ books    ┆ ID       │
│ 85d63b71-2463-4097-adfd-012c23… ┆ scroll      ┆ android   ┆ browser ┆ books    ┆ ID       │
│ 85d63b71-2463-4097-adfd-012c23… ┆ viewcontent ┆ android   ┆ browser ┆ beauty   ┆ ID       │
│ 85d63b71-2463-4097-adfd-012c23… ┆ add_to_cart ┆ android   ┆ browser ┆ books    ┆ ID       │
└─────────────────────────────────┴─────────────┴───────────┴─────────┴──────────┴──────────┘

In [ ]:
df_pd[df_pd["cluster"]==1]

In [ ]:
df_pd.groupby("muid").value_counts().to_dict()# Assume your dataframe has columns: muid, cluster
result = (
    df_pd.groupby("muid")
    .agg(lambda x: x.value_counts().index[0])   # pick most frequent
    .reset_index()
)
result

In [24]:
import pickle
with open(r"C:\Users\muthuvu\Videos\10X-MLE-master\kmodes_model.pkl", "wb") as f:
    pickle.load(f)

UnsupportedOperation: read

In [30]:


import pickle


with open(r"C:\Users\muthuvu\Videos\10X-MLE-master\kmodes_model.pkl", "rb") as f:
    pickle.load(f)

EOFError: Ran out of input

In [ ]:
import pandas
chunksize = 1e4  # 1 million
results = []

for chunk in events.collect():
    preds = km.predict(chunk)
    results.extend(preds)   # or save directly to disk


In [ ]:

pl.scan_parquet("/home/muthu/Desktop/New Folder/data/events/part-00001.parquet").fetch()#.select(pl.col("location").value_counts()).collect()

In [26]:
p = r"C:\Users\muthuvu\Videos\10X-MLE-master\notebook\output_clusters_single\part.0.parquet"
cluster_events_pd =pl.scan_parquet(p).collect().to_pandas()
cluster_events_pd = cluster_events_pd[cluster_events_pd.duplicated()].reset_index(drop=True)
cluster_events_pd
                                                       

muid   event_name device_os  \
0         f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android   
1         f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android   
2         f2ffe5a9-165d-4021-8707-d197d7ce8e20     pageview   android   
3         f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android   
4         f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android   
...                                        ...          ...       ...   
10483072  85d63b71-2463-4097-adfd-012c236bb94a     pageview   android   
10483073  85d63b71-2463-4097-adfd-012c236bb94a  viewcontent   android   
10483074  85d63b71-2463-4097-adfd-012c236bb94a       scroll   android   
10483075  85d63b71-2463-4097-adfd-012c236bb94a  viewcontent   android   
10483076  85d63b71-2463-4097-adfd-012c236bb94a  add_to_cart   android   

          channel category location  cluster  
0         browser    books       CA        0  
1         browser    books       CA        0  
2         browser    books       CA       19  
3         browser  fashion       CA       60  
4         browser    books       CA        0  
...           ...      ...      ...      ...  
10483072  browser    books       ID       19  
10483073  browser    books       ID       16  
10483074  browser    books       ID        0  
10483075  browser   beauty       ID       74  
10483076  browser    books       ID       26  

[10483077 rows x 7 columns]

In [61]:
# Reset index so we have normal columns
per_user_pd = cluster_events_pd.groupby("muid").value_counts(["cluster"]).reset_index(name="count")

# For each user, pick top 4 clusters
top4_per_user = (
    per_user_pd
    .sort_values(["muid", "count"], ascending=[True, False])
    .groupby("muid")
    .head(4)
)

top4_per_user


muid  cluster  count
0       000122b6-1f4e-4783-a220-719baa57890f       15      9
1       000122b6-1f4e-4783-a220-719baa57890f        4      8
2       000122b6-1f4e-4783-a220-719baa57890f        9      7
3       000122b6-1f4e-4783-a220-719baa57890f       24      7
9       0002d5fb-4157-422d-a76c-beb53201c00b        0     14
...                                      ...      ...    ...
962245  fffce386-c870-4f97-8827-d257e0e2c650        3      6
962250  fffe3f6b-0acf-45d2-a961-f6316d9c2e7f        6     38
962251  fffe3f6b-0acf-45d2-a961-f6316d9c2e7f       57     27
962252  fffe3f6b-0acf-45d2-a961-f6316d9c2e7f       44     26
962253  fffe3f6b-0acf-45d2-a961-f6316d9c2e7f       91     10

[341237 rows x 3 columns]

In [64]:
top4_per_user[top4_per_user["muid"] == top4_per_user["muid"][962251]]
top4_per_user.to_csv(r"C:\Users\muthuvu\Videos\10X-MLE-master\notebook\output_clusters_single\per_user_top4_clsuters.csv", index=False)

In [65]:
top4_per_user

muid  cluster  count
0       000122b6-1f4e-4783-a220-719baa57890f       15      9
1       000122b6-1f4e-4783-a220-719baa57890f        4      8
2       000122b6-1f4e-4783-a220-719baa57890f        9      7
3       000122b6-1f4e-4783-a220-719baa57890f       24      7
9       0002d5fb-4157-422d-a76c-beb53201c00b        0     14
...                                      ...      ...    ...
962245  fffce386-c870-4f97-8827-d257e0e2c650        3      6
962250  fffe3f6b-0acf-45d2-a961-f6316d9c2e7f        6     38
962251  fffe3f6b-0acf-45d2-a961-f6316d9c2e7f       57     27
962252  fffe3f6b-0acf-45d2-a961-f6316d9c2e7f       44     26
962253  fffe3f6b-0acf-45d2-a961-f6316d9c2e7f       91     10

[341237 rows x 3 columns]

In [68]:
final_cluster_pd = (
    cluster_events_pd
    .merge(
        top4_per_user[["muid", "cluster"]],   # keep only the keys we care about
        on=["muid", "cluster"],               # join on both columns
        how="inner"                           # keep only matches
    )
)
final_cluster_pd


muid   event_name device_os  channel  \
0        f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
1        f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
2        f2ffe5a9-165d-4021-8707-d197d7ce8e20     pageview   android  browser   
3        f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
4        f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
...                                       ...          ...       ...      ...   
7802231  85d63b71-2463-4097-adfd-012c236bb94a  viewcontent   android  browser   
7802232  85d63b71-2463-4097-adfd-012c236bb94a     pageview   android  browser   
7802233  85d63b71-2463-4097-adfd-012c236bb94a  viewcontent   android  browser   
7802234  85d63b71-2463-4097-adfd-012c236bb94a       scroll   android  browser   
7802235  85d63b71-2463-4097-adfd-012c236bb94a  add_to_cart   android  browser   

        category location  cluster  
0          books       CA        0  
1          books       CA        0  
2          books       CA       19  
3        fashion       CA       60  
4          books       CA        0  
...          ...      ...      ...  
7802231    books       ID       16  
7802232    books       ID       19  
7802233    books       ID       16  
7802234    books       ID        0  
7802235    books       ID       26  

[7802236 rows x 7 columns]

In [72]:
final_cluster_pd.drop_duplicates(inplace=True)

In [84]:
final_cluster_pd.to_csv(r"C:\Users\muthuvu\Videos\10X-MLE-master\notebook\output_clusters_single\per_user_top4_clsuters_noduplicates.csv", index=False)

In [82]:
final_cluster_pd.groupby("cluster").count().sort_values(by="muid", ascending=False)

muid  event_name  device_os  channel  category  location
cluster                                                           
1        55242       55242      55242    55242     55242     55242
4        42007       42007      42007    42007     42007     42007
0        39264       39264      39264    39264     39264     39264
14       33533       33533      33533    33533     33533     33533
6        33326       33326      33326    33326     33326     33326
...        ...         ...        ...      ...       ...       ...
94           2           2          2        2         2         2
142          2           2          2        2         2         2
141          2           2          2        2         2         2
133          1           1          1        1         1         1
134          1           1          1        1         1         1

[145 rows x 6 columns]

In [85]:
final_cluster_pd.to_parquet(r"C:\Users\muthuvu\Videos\10X-MLE-master\notebook\output_clusters_single\per_user_top4_clsuters_noduplicates.parquet")

In [95]:
final_cluster_pd.reset_index(drop=True, inplace=True)

In [99]:
final_cluster_pd["summary_per_user"] = final_cluster_pd.apply(
    lambda x: f"In {x['location']}, the user purchased {x['category']} during {x['event_name']} using {x['channel']} on {x['device_os']}",
    axis=1
)

final_cluster_pd.to_parquet(r"C:\Users\muthuvu\Videos\10X-MLE-master\notebook\output_clusters_single\per_user_top4_clsuters_noduplicates_summary_peruser.parquet")


In [128]:
s = ". ".join(final_cluster_pd[final_cluster_pd["cluster"]==0]["summary_per_user"].values)
s[:500]

'In CA, the user purchased books during scroll using browser on android. In IN, the user purchased books during viewcontent using browser on android. In IN, the user purchased books during scroll using browser on android. In IN, the user purchased books during purchase using browser on android. In IN, the user purchased books during pageview using browser on android. In IN, the user purchased coffee during scroll using browser on android. In IN, the user purchased books during add_to_cart using b'

In [131]:
final_cluster_pd[final_cluster_pd["cluster"]==0]

muid   event_name device_os  channel  \
0       f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
36      ce204111-f584-4d25-8ec1-8c0ca41e64f6  viewcontent   android  browser   
37      ce204111-f584-4d25-8ec1-8c0ca41e64f6       scroll   android  browser   
38      ce204111-f584-4d25-8ec1-8c0ca41e64f6     purchase   android  browser   
39      ce204111-f584-4d25-8ec1-8c0ca41e64f6     pageview   android  browser   
...                                      ...          ...       ...      ...   
819315  ce605607-c7e7-417a-91e1-031806277d5e       scroll   android  browser   
819316  ce605607-c7e7-417a-91e1-031806277d5e       scroll   android  browser   
819385  cd3f2533-3c7d-4838-8d82-8be4ef9ce9f9       scroll   android  browser   
819413  e5674540-3569-4bd0-b6c0-b6adb72c9ffe       scroll   android  browser   
819464  85d63b71-2463-4097-adfd-012c236bb94a       scroll   android  browser   

       category location  cluster  \
0         books       CA        0   
36        books       IN        0   
37        books       IN        0   
38        books       IN        0   
39        books       IN        0   
...         ...      ...      ...   
819315    books       BD        0   
819316   laptop       BD        0   
819385    books       UK        0   
819413    books       US        0   
819464    books       ID        0   

                                         summary_per_user  
0       In CA, the user purchased books during scroll ...  
36      In IN, the user purchased books during viewcon...  
37      In IN, the user purchased books during scroll ...  
38      In IN, the user purchased books during purchas...  
39      In IN, the user purchased books during pagevie...  
...                                                   ...  
819315  In BD, the user purchased books during scroll ...  
819316  In BD, the user purchased laptop during scroll...  
819385  In UK, the user purchased books during scroll ...  
819413  In US, the user purchased books during scroll ...  
819464  In ID, the user purchased books during scroll ...  

[39264 rows x 8 columns]

In [ ]:
for i in 
cluster_0 = final_cluster_pd[final_cluster_pd["cluster"]==2]
cluster_0[['event_name', 'device_os', 'channel', 'category']].mode()

NameError: name 'final_cluster_pd' is not defined

In [147]:
cluster_0[
    (cluster_0['event_name']=='pageview') & 
    (cluster_0['device_os']=='android') &
    (cluster_0['channel']=='app') & 
    (cluster_0['category']=='coffee')
].shape[0]/cluster_0.shape[0]

0.22610213880401572

In [142]:
cluster_summaries = final_cluster_pd[final_cluster_pd["cluster"]==0]["summary_per_user"].values

chunk_size = 100  # number of summaries per chunk
chunks = [" ".join(cluster_summaries[i:i+chunk_size]) for i in range(0, len(cluster_summaries), chunk_size)]

chunk_summaries = []

for chunk in chunks:
    inputs = tokenizer("summarize: " + chunk, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        inputs["input_ids"], 
        max_length=80, 
        min_length=30, 
        length_penalty=2.0, 
        num_beams=4, 
        early_stopping=True
    )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    chunk_summaries.append(summary)

KeyboardInterrupt: 

In [129]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

paragraph = s
# Encode input
inputs = tokenizer("summarize: " + paragraph, return_tensors="pt", max_length=100000, truncation=True)

# Generate summary
outputs = model.generate(inputs["input_ids"], max_length=80, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(summary)


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 320000000000 bytes.